导入相关包

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
file_path = 'E:/course/大三秋/数据科学导引/input'
os.listdir(file_path)

['sampleSubmission.csv', 'test.tsv.zip', 'train.tsv.zip']

In [3]:
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

读取数据

In [6]:
train=pd.read_csv('E:/course/大三秋/数据科学导引/input/train.tsv', sep='\t',dtype={"index":int,"sentence":int,"phrase":str,"sentiment":int})
test=pd.read_csv('E:/course/大三秋/数据科学导引/input/test.tsv', sep='\t',dtype={"index":int,"sentence":int,"phrase":str,"sentiment":int})
sample_sub = pd.read_csv('E:/course/大三秋/数据科学导引/input/sampleSubmission.csv',dtype={"index":int,"sentence":int,"phrase":str,"sentiment":int})

查看数据情况

In [7]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [8]:
test

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


使用词袋模型向量化

In [9]:
# 将短语和标签提取出来
X = train['Phrase']  # 短语
y = train['Sentiment']  # 情感标签

# 将数据拆分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 文本向量化
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

In [ ]:
XGBoost模型

In [10]:
X = X_train_vec
y = y_train

使用XGBoost训练模型

In [24]:
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                     learning_rate=0.01,  
                    colsample_bytree = 0.9,
                    subsample = 0.8,
                   n_estimators=5000, 
                  reg_alpha = 0.3,
                      max_depth=4, 
                       gamma=1,random_state=42)

In [32]:
import optuna

In [33]:
# 定义目标函数，用于Optuna调参
def objective(trial):
    # 为XGBClassifier的参数进行采样
    params = {
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'random_state': 42
    }
    
    # 创建XGBClassifier实例
    xgb = XGBClassifier(
        silent=False,
        **params
    )
    
    # 训练模型
    xgb.fit(X_train_vec, y_train)
    
    # 预测测试集
    predictions = xgb.predict(X_val_vec)
    
    # 计算准确率
    accuracy = accuracy_score(y_val, predictions)
    
    # 返回准确率，Optuna将根据此值进行优化
    return accuracy

In [ ]:
%%time
#使用optuna调参
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# 输出最佳参数和结果
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print(" Value: {}".format(trial.value))
print(" Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key, value))

[I 2024-11-24 10:09:56,628] A new study created in memory with name: no-name-b2ed9992-d6bf-4c42-af55-cf6309cb7929


In [ ]:
%%time
xgb.fit(X_train_vec,y_train)

CPU times: total: 54min 16s
Wall time: 28min 22s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
%%time
# Prediction on validation set
y_pred = xgb.predict(X_val_vec)

# Evaluating accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
#classification report
report = classification_report(y_val, y_pred)
print(report)

Validation Accuracy: 0.5780148660771498
CPU times: total: 2min 38s
Wall time: 34 s


In [16]:
nan_mask = pd.isnull(X_test)  # 或者使用 pd.isna(X_test)
if nan_mask.any().any():  # 检查是否有任何NaN值
    print("存在NaN值")
else:
    print("不存在NaN值")

存在NaN值


In [27]:
X_test=test['Phrase']
X_test_vec =vectorizer.transform(X_test)

In [28]:
#填补缺失值
X_test.fillna('<UNK>', inplace=True)

In [29]:
%%time
# Prediction on test set
xgb_predict=xgb.predict(X_test_vec)

CPU times: total: 5min 24s
Wall time: 1min 4s


In [20]:
xgb_predict

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

提交测试结果

In [30]:
submission_file =pd.read_csv("E:\course\大三秋\数据科学导引\input\sampleSubmission.csv",sep=',')
submission_file['Sentiment']=xgb_predict
submission_file

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,2
66288,222349,2
66289,222350,2
66290,222351,2


In [31]:
submission_file.to_csv('Submission_XGB.csv',index=False)